### Beginning of the Assignment - exploration

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
#pd.set_option('max_columns', 200)

In [43]:
character_nicknames_df = pd.read_csv('datasets/PHIL DATA SET/character_nicknames.csv')

In [7]:
details_df = pd.read_csv('datasets/PHIL DATA SET/details.csv')

In [17]:
favs_df = pd.read_csv('datasets/PHIL DATA SET/favs.csv')


In [60]:
person_details_df = pd.read_csv('datasets/PHIL DATA SET/person_details.csv')


In [31]:
person_alternate_names_df = pd.read_csv('datasets/PHIL DATA SET/person_alternate_names.csv')


In [68]:
person_anime_works_df = pd.read_csv('datasets/PHIL DATA SET/person_anime_works.csv')


In [73]:
stats_df = pd.read_csv('datasets/PHIL DATA SET/stats.csv')


##### First look

In [43]:
character_nicknames_df.shape

(37080, 2)

In [44]:
character_nicknames_df.head()

,character_mal_id,nickname
0,280205,Hikaruko
1,280129,Hinacchi
2,280127,Bertha Willis
3,280066,Jimmy
4,280059,Full Body Red Square


In [45]:
character_nicknames_df.columns
#will list all the columns. Not necessary here but kept as a pattern to follow with the following files

Index(['character_mal_id', 'nickname'], dtype='object')

In [46]:
character_nicknames_df.dtypes

character_mal_id     int64
nickname            object
dtype: object

In [47]:
character_nicknames_df.describe()
#not needed here because it should just be ids

,character_mal_id
count,37080.000000
mean,115767.769579
std,87596.063426
min,3.000000
25%,30541.750000
50%,109656.000000
75%,192520.750000
max,282255.000000


### Data preparation (cleaning)


##### On the first dataset we may need to check for duplicates.
What does that mean? We have 102 rows that are duplicated over a 37080 rows dataset.
Why is that? Are there some characters that have multiple nicknames, so they are repeated in the dataset?

In [15]:
character_nicknames_df.loc[character_nicknames_df.duplicated()]
#by default will give us the second

,character_mal_id,nickname


In [14]:
character_nicknames_df.query('character_mal_id == 75628')

,character_mal_id,nickname
21503,75628,Anneroze
21504,75628,Witch of Steel
21505,75628,Iron Witch


Mhh they're somehow different so yeah, the same character could have different nicknames.
We want to drop though the ones that are exactly the same.

In [13]:
#this way we drop the duplicates on the first dataset

character_nicknames_df = character_nicknames_df\
    .loc[~character_nicknames_df.duplicated()]\
    .reset_index(drop=True).copy()

#we don't need to use a subset here because there are just 2 columns

Let's check for nan values

In [46]:
character_nicknames_df[character_nicknames_df.isna().any(axis=1)]


,character_mal_id,nickname


In [45]:
#cleaning the df from nan values
character_nicknames_df = character_nicknames_df.dropna()


##### On the second dataset we may need to check for missing values and/or inconsistent values, since there are no duplicates

In [81]:
details_df.isna().sum()

mal_id                 0
title                  0
title_japanese       119
url                    0
image_url              0
type                  67
status                 0
score              10073
scored_by          10073
start_date           851
end_date           17688
synopsis            5127
rank                6958
popularity             0
members                0
favorites              0
genres                 0
studios                0
themes                 0
demographics           0
source                 0
rating               600
episodes             680
season             22689
year               22689
producers              0
explicit_genres        0
licensors              0
streaming              0
dtype: int64

In [28]:
details_df
#anime details
#japanes title could be dropped?
#members stand for how many users have this anime added to their list.
#explicit_genres is empty so can be removed
#licensor and streaming are mostly empty. Do we care?

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
0,59356,-Socket-,-socket-,https://myanimelist.net/anime/59356/-Socket-,https://cdn.myanimelist.net/images/anime/1043/...,Movie,Finished Airing,NaN,NaN,2010-01-01T00:00:00+00:00,...,[],Original,G - All Ages,1.0,NaN,NaN,['Nagoya Zokei University'],[],[],[]
1,56036,......,......,https://myanimelist.net/anime/56036/-,https://cdn.myanimelist.net/images/anime/1057/...,Music,Finished Airing,6.53,503.0,2023-06-11T00:00:00+00:00,...,[],Original,PG-13 - Teens 13 or older,1.0,NaN,NaN,[],[],[],[]
2,2928,.hack//G.U. Returner,.HACK//G.U. RETURNER,https://myanimelist.net/anime/2928/hack__GU_Re...,https://cdn.myanimelist.net/images/anime/1798/...,OVA,Finished Airing,6.65,9745.0,2007-01-18T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,"['Bandai Visual', 'CyberConnect2']",[],[],[]
3,3269,.hack//G.U. Trilogy,.hack//G.U. Trilogy,https://myanimelist.net/anime/3269/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/1566/...,Movie,Finished Airing,7.06,15373.0,2007-12-22T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],"['Funimation', 'Bandai Entertainment']",[]
4,4469,.hack//G.U. Trilogy: Parody Mode,.hack//G.U. Trilogy,https://myanimelist.net/anime/4469/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/10/86...,Special,Finished Airing,6.35,4317.0,2008-03-25T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28950,59421,Zutaboro Reijou wa Ane no Moto Konyakusha ni D...,ずたぼろ令嬢は姉の元婚約者に溺愛される,https://myanimelist.net/anime/59421/Zutaboro_R...,https://cdn.myanimelist.net/images/anime/1518/...,TV,Finished Airing,7.37,15624.0,2025-07-05T00:00:00+00:00,...,['Josei'],Light novel,PG-13 - Teens 13 or older,12.0,summer,2025.0,"['Studio Pierrot', 'Mainichi Broadcasting Syst...",[],[],"['Crunchyroll', 'Aniplus TV', 'Bahamut Anime C..."
28951,31245,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～,https://myanimelist.net/anime/31245/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/3/821...,Movie,Finished Airing,7.20,104106.0,2016-04-23T00:00:00+00:00,...,[],Music,PG-13 - Teens 13 or older,1.0,NaN,NaN,"['Aniplex', 'Dentsu', 'Kadokawa Shoten', 'Movi...",[],['Aniplex of America'],[]
28952,36305,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～ 「金曜日のおはよう」,https://myanimelist.net/anime/36305/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/6/883...,Special,Finished Airing,7.17,10038.0,2016-10-26T00:00:00+00:00,...,[],Music,PG - Children,1.0,NaN,NaN,['Aniplex'],[],[],[]
28953,34895,Zutto Suki Datta,ずっと好きだった,https://myanimelist.net/anime/34895/Zutto_Suki...,https://cdn.myanimelist.net/images/anime/1498/...,OVA,Finished Airing,5.68,1887.0,2017-04-21T00:00:00+00:00,...,[],Manga,Rx - Hentai,2.0,NaN,NaN,"['Queen Bee', 'Mediabank']",[],[],[]


In [13]:
details_df[['start_date','season']].query("season.notna()")
#season can be removed? Do we care about the season? We can "calculate" it from the "start_date" field

,start_date,season
10,2006-04-06T00:00:00+00:00,spring
11,2002-04-04T00:00:00+00:00,spring
12,2003-01-09T00:00:00+00:00,winter
19,2024-07-07T00:00:00+00:00,summer
20,2025-01-05T00:00:00+00:00,winter
...,...,...
28906,2001-04-07T00:00:00+00:00,spring
28913,2011-05-18T00:00:00+00:00,spring
28932,1980-04-15T00:00:00+00:00,spring
28939,2012-04-03T00:00:00+00:00,spring


In [5]:
details_df.query("episodes > 2500")

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
13697,9947,Lan Mao,蓝猫淘气3000问,https://myanimelist.net/anime/9947/Lan_Mao,https://cdn.myanimelist.net/images/anime/5/275...,TV,Finished Airing,6.06,228.0,1999-10-08T00:00:00+00:00,...,[],Original,PG - Children,3000.0,NaN,NaN,['Beijing Sunchime Happy Culture Company'],[],[],[]


In [23]:
details_df.dtypes
#scoredby, rank, episodes, year can be an int instead of a float
#start and end dates are not objects but dates
#do we need to swap the empty [] with Nan or not? WE should in order to be able to use the .isna() method and other pandas methods


mal_id               int64
title               object
title_japanese      object
url                 object
image_url           object
type                object
status              object
score              float64
scored_by          float64
start_date          object
end_date            object
synopsis            object
rank               float64
popularity           int64
members              int64
favorites            int64
genres              object
studios             object
themes              object
demographics        object
source              object
rating              object
episodes           float64
season              object
year               float64
producers           object
explicit_genres     object
licensors           object
streaming           object
dtype: object

THIRD DATASET

In [18]:
favs_df

,username,fav_type,id
0,ishikawas,anime,45649
1,ishikawas,anime,38680
2,ishikawas,anime,795
3,ishikawas,anime,37510
4,ishikawas,anime,820
...,...,...,...
4178742,vincent0607,character,497
4178743,vincent0607,character,118739
4178744,vincent0607,character,188177
4178745,vincent0607,character,141354


In [19]:
favs_df.isna().sum()


username    4
fav_type    0
id          0
dtype: int64

In [22]:
favs_df.dtypes

username    object
fav_type    object
id           int64
dtype: object

## FOURTH DATASET

In [32]:
person_alternate_names_df

,person_mal_id,alt_name
0,1,Seki Mondoya
1,1,門戸 開
2,1,Monto Hiraku
3,3,雪野五月
4,10,Kevin Hatcher
...,...,...
20460,89567,Sydsnap
20461,89567,Queen of Degeneracy
20462,89826,陳浩
20463,89842,Chidori


In [38]:
person_alternate_names_df.dtypes

person_mal_id     int64
alt_name         object
dtype: object

In [48]:
person_alternate_names_df.isna().sum()

person_mal_id    0
alt_name         0
dtype: int64

In [47]:
person_alternate_names_df = person_alternate_names_df.dropna()


## FIFTH DATASET

In [49]:
#let's check for duplicates

In [62]:
person_details_df.loc[person_details_df['person_mal_id'].duplicated()]
#we found that the duplicates differ for the "relevant_location" field, which has no interest for us so we drop the duplicates
#TODO

,person_mal_id,url,website_url,image_url,name,given_name,family_name,birthday,favorites,relevant_location
4171,4564,https://myanimelist.net/people/4564/Natsuko_Ta...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Natsuko Takahashi,ナツコ,高橋,NaN,19,"Paris, France"
4172,4564,https://myanimelist.net/people/4564/Natsuko_Ta...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Natsuko Takahashi,ナツコ,高橋,NaN,19,"Cape Town, South Africa"
5202,5742,https://myanimelist.net/people/5742/Sarah_Strange,NaN,https://cdn.myanimelist.net/images/voiceactors...,Sarah Strange,NaN,NaN,NaN,11,"Yokohama, Japan"
6018,6638,https://myanimelist.net/people/6638/Toshio_Suzuki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Toshio Suzuki,敏夫,鈴木,1948-08-19T00:00:00+00:00,354,"Paris, France"
6019,6638,https://myanimelist.net/people/6638/Toshio_Suzuki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Toshio Suzuki,敏夫,鈴木,1948-08-19T00:00:00+00:00,354,"Mumbai, India"
...,...,...,...,...,...,...,...,...,...,...
64387,77565,https://myanimelist.net/people/77565/Ena_Nishi...,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Ena Nishikawa,絵奈,西川,NaN,0,"Paris, France"
64402,77579,https://myanimelist.net/people/77579/Amina_Gaede,https://www.instagram.com/amina.gaede/,https://cdn.myanimelist.net/images/voiceactors...,Amina Gaede,NaN,NaN,1998-07-05T00:00:00+00:00,0,"São Paulo, Brazil"
64412,77588,https://myanimelist.net/people/77588/Willie_Ra...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Willie Ray Jr.,NaN,NaN,NaN,0,"Berlin, Germany"
64414,77589,https://myanimelist.net/people/77589/Brandi_Ray,NaN,https://cdn.myanimelist.net/images/voiceactors...,Brandi Ray,NaN,NaN,NaN,0,"Mumbai, India"


In [63]:
person_details_df[person_details_df.isna().any(axis=1)]


,person_mal_id,url,website_url,image_url,name,given_name,family_name,birthday,favorites,relevant_location
0,1,https://myanimelist.net/people/1/Tomokazu_Seki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Tomokazu Seki,智一,関,1972-09-08T00:00:00+00:00,6219,"Berlin, Germany"
2,3,https://myanimelist.net/people/3/Satsuki_Yukino,NaN,https://cdn.myanimelist.net/images/voiceactors...,Satsuki Yukino,さつき,ゆきの,1970-05-25T00:00:00+00:00,1777,"Madrid, Spain"
9,10,https://myanimelist.net/people/10/Johnny_Yong_...,https://johnnyyongbosch.com/,https://cdn.myanimelist.net/images/voiceactors...,Johnny Yong Bosch,NaN,NaN,1976-01-06T00:00:00+00:00,12869,"Los Angeles, USA"
10,11,https://myanimelist.net/people/11/Kouichi_Yama...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Kouichi Yamadera,宏一,山寺,1961-06-17T00:00:00+00:00,1574,"New York, USA"
11,12,https://myanimelist.net/people/12/Steven_Blum,http://www.steveblumvoices.com,https://cdn.myanimelist.net/images/voiceactors...,Steven Blum,NaN,NaN,1960-04-29T00:00:00+00:00,7749,"São Paulo, Brazil"
...,...,...,...,...,...,...,...,...,...,...
76694,90011,https://myanimelist.net/people/90011/Nanako_Ki...,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Nanako Kishimoto,七子,岸本,NaN,0,"Mumbai, India"
76695,90012,https://myanimelist.net/people/90012/Pamon,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Pamon,NaN,파몬,NaN,0,"Tokyo, Japan"
76696,90013,https://myanimelist.net/people/90013/Tomoru_Emoto,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Tomoru Emoto,ともる,柄本,NaN,0,"Tokyo, Japan"
76697,90014,https://myanimelist.net/people/90014/Hirari,https://hirari.2-d.jp/,https://cdn.myanimelist.net/img/sp/icon/apple-...,Hirari,NaN,ひらり,NaN,0,"Paris, France"


In [64]:
person_details_df.dtypes
#we need to change birthday from object to data

person_mal_id         int64
url                  object
website_url          object
image_url            object
name                 object
given_name           object
family_name          object
birthday             object
favorites             int64
relevant_location    object
dtype: object

In [67]:
person_details_df.isna().sum()
#we have to check the nan values

person_mal_id            0
url                      0
website_url          59264
image_url                0
name                     2
given_name           30253
family_name          18641
birthday             59461
favorites                0
relevant_location        0
dtype: int64

We can join the two tables person_details_df and person_alternate_names_df having the keys that match.
Putting the alternate names in a new column called alt_name and having a list of those inside

### SIXTH DATASET

In [69]:
person_anime_works_df

,person_mal_id,position,anime_mal_id
0,1,Theme Song Performance,3080
1,1,Inserted Song Performance,15699
2,1,Theme Song Performance (OP),247
3,1,Theme Song Performance,258
4,1,Theme Song Performance (ED),34825
...,...,...,...
458086,89951,In-Between Animation,11001
458087,89951,Key Animation,55092
458088,89951,"Key Animation (eps 4, 9)",20053
458089,89951,Key Animation,50553


In [72]:
person_anime_works_df.dtypes
#the types are correct

person_mal_id     int64
position         object
anime_mal_id      int64
dtype: object

In [71]:
person_anime_works_df.isna().sum()
#There's no nan value

person_mal_id    0
position         0
anime_mal_id     0
dtype: int64

In [74]:
stats_df

,mal_id,watching,completed,on_hold,dropped,plan_to_watch,total,score_1_votes,score_1_percentage,score_2_votes,...,score_6_votes,score_6_percentage,score_7_votes,score_7_percentage,score_8_votes,score_8_percentage,score_9_votes,score_9_percentage,score_10_votes,score_10_percentage
0,59356,7,146,4,20,20,197,2.0,2.2,0.0,...,33.0,36.3,19.0,20.9,2.0,2.2,0.0,0.0,1.0,1.1
1,56036,21,770,8,29,113,941,5.0,1.0,6.0,...,138.0,27.4,144.0,28.6,81.0,16.1,17.0,3.4,40.0,8.0
2,2928,451,14953,302,349,6472,22527,101.0,1.0,93.0,...,2054.0,21.1,2709.0,27.8,1500.0,15.4,875.0,9.0,608.0,6.2
3,3269,726,22790,452,537,9762,34267,120.0,0.8,156.0,...,2457.0,16.0,4157.0,27.0,3075.0,20.0,1919.0,12.5,1400.0,9.1
4,4469,241,6918,182,266,3528,11135,83.0,1.9,104.0,...,888.0,20.6,871.0,20.2,592.0,13.7,308.0,7.1,315.0,7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28950,59421,11349,17353,574,1440,15729,46445,54.0,0.3,40.0,...,2303.0,14.5,5545.0,34.9,3919.0,24.7,1598.0,10.1,1235.0,7.8
28951,31245,10332,140676,2989,1416,94249,249662,357.0,0.3,484.0,...,15972.0,15.3,34370.0,33.0,24387.0,23.4,10480.0,10.1,7589.0,7.3
28952,36305,928,16119,388,243,9656,27334,48.0,0.5,28.0,...,1839.0,18.3,3258.0,32.5,2009.0,20.0,887.0,8.8,936.0,9.3
28953,34895,896,2387,375,356,1512,5526,116.0,6.1,80.0,...,437.0,23.2,280.0,14.8,170.0,9.0,68.0,3.6,125.0,6.6


In [75]:
stats_df.dtypes
#we can swap *_votes to int instead of floats

mal_id                   int64
watching                 int64
completed                int64
on_hold                  int64
dropped                  int64
plan_to_watch            int64
total                    int64
score_1_votes          float64
score_1_percentage     float64
score_2_votes          float64
score_2_percentage     float64
score_3_votes          float64
score_3_percentage     float64
score_4_votes          float64
score_4_percentage     float64
score_5_votes          float64
score_5_percentage     float64
score_6_votes          float64
score_6_percentage     float64
score_7_votes          float64
score_7_percentage     float64
score_8_votes          float64
score_8_percentage     float64
score_9_votes          float64
score_9_percentage     float64
score_10_votes         float64
score_10_percentage    float64
dtype: object

In [78]:
stats_df.isna().sum()
#there are 430 series without any vote (?)
#we need to understand which ones are they

mal_id                   0
watching                 0
completed                0
on_hold                  0
dropped                  0
plan_to_watch            0
total                    0
score_1_votes          430
score_1_percentage     430
score_2_votes          430
score_2_percentage     430
score_3_votes          430
score_3_percentage     430
score_4_votes          430
score_4_percentage     430
score_5_votes          430
score_5_percentage     430
score_6_votes          430
score_6_percentage     430
score_7_votes          430
score_7_percentage     430
score_8_votes          430
score_8_percentage     430
score_9_votes          430
score_9_percentage     430
score_10_votes         430
score_10_percentage    430
dtype: int64

In [80]:
stats_df[stats_df['score_1_votes'].isna()]


,mal_id,watching,completed,on_hold,dropped,plan_to_watch,total,score_1_votes,score_1_percentage,score_2_votes,...,score_6_votes,score_6_percentage,score_7_votes,score_7_percentage,score_8_votes,score_8_percentage,score_9_votes,score_9_percentage,score_10_votes,score_10_percentage
124,62496,6,0,2,0,652,660,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,61637,0,0,0,0,2128,2128,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,62002,1,0,0,0,1504,1505,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,61925,3,0,1,1,11270,11275,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437,60917,0,0,0,0,739,739,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28835,61947,0,0,0,0,59,59,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28859,61646,0,0,0,0,120,120,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28860,61640,0,0,0,0,131,131,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28898,57990,0,0,0,0,2308,2308,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
details_df.query('mal_id == 61558')

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
529,61558,Aikatsu! x PriPara the Movie: Deai no Kiseki!,アイカツ！×プリパラ THE MOVIE -出会いのキセキ！-,https://myanimelist.net/anime/61558/Aikatsu_x_...,https://cdn.myanimelist.net/images/anime/1723/...,Movie,Not yet aired,NaN,NaN,2025-10-10T00:00:00+00:00,...,[],Original,NaN,1.0,NaN,NaN,[],[],[],[]
